# Vector Autoregression (VAR)

In [16]:
from IPython.display import display, Markdown
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error

# Air Quality Dataset

In [2]:
# Load in the dataset
data = pd.read_csv("AirQualityUCI.csv", sep=';')

In [3]:
# Clean empty columns
data = data.drop(['Unnamed: 15', 'Unnamed: 16'], axis=1)

In [4]:
# Clean empty rows
data = data.dropna(axis=0)

In [5]:
def report(df):
    display(Markdown('<b>head():</b>'))
    display(df.head())
    display(Markdown('<b>describe():</b>'))
    display(df.describe())
    display(Markdown('<b>info():</b>'))
    display(df.info(verbose=True))
    display(Markdown('<b>infer_dtype():</b>'))
    display(df.apply(lambda x: pd.api.types.infer_dtype(x.values)))

In [6]:
report(data)

<b>head():</b>

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"


<b>describe():</b>

,PT08.S1(CO),NMHC(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3)
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,1048.990061,-159.090093,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032
std,329.832710,139.789093,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,921.000000,-200.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000
50%,1053.000000,-200.000000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000
75%,1221.000000,-200.000000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000
max,2040.000000,1189.000000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000


<b>info():</b>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9357 entries, 0 to 9356
Data columns (total 15 columns):
Date             9357 non-null object
Time             9357 non-null object
CO(GT)           9357 non-null object
PT08.S1(CO)      9357 non-null float64
NMHC(GT)         9357 non-null float64
C6H6(GT)         9357 non-null object
PT08.S2(NMHC)    9357 non-null float64
NOx(GT)          9357 non-null float64
PT08.S3(NOx)     9357 non-null float64
NO2(GT)          9357 non-null float64
PT08.S4(NO2)     9357 non-null float64
PT08.S5(O3)      9357 non-null float64
T                9357 non-null object
RH               9357 non-null object
AH               9357 non-null object
dtypes: float64(8), object(7)
memory usage: 1.1+ MB


None

<b>infer_dtype():</b>

Date               string
Time               string
CO(GT)             string
PT08.S1(CO)      floating
NMHC(GT)         floating
C6H6(GT)           string
PT08.S2(NMHC)    floating
NOx(GT)          floating
PT08.S3(NOx)     floating
NO2(GT)          floating
PT08.S4(NO2)     floating
PT08.S5(O3)      floating
T                  string
RH                 string
AH                 string
dtype: object

In [8]:
data['CO(GT)'] = data['CO(GT)'].str.replace(',', '.').astype(float)
data['C6H6(GT)'] = data['C6H6(GT)'].str.replace(',','.').astype(float)
data['T'] = data['T'].str.replace(',', '.').astype(float)
data['RH'] = data['RH'].str.replace(',', '.').astype(float)
data['AH'] = data['AH'].str.replace(',', '.').astype(float)

In [9]:
report(data)

<b>head():</b>

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


<b>describe():</b>

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


<b>info():</b>

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9357 entries, 0 to 9356
Data columns (total 15 columns):
Date             9357 non-null object
Time             9357 non-null object
CO(GT)           9357 non-null float64
PT08.S1(CO)      9357 non-null float64
NMHC(GT)         9357 non-null float64
C6H6(GT)         9357 non-null float64
PT08.S2(NMHC)    9357 non-null float64
NOx(GT)          9357 non-null float64
PT08.S3(NOx)     9357 non-null float64
NO2(GT)          9357 non-null float64
PT08.S4(NO2)     9357 non-null float64
PT08.S5(O3)      9357 non-null float64
T                9357 non-null float64
RH               9357 non-null float64
AH               9357 non-null float64
dtypes: float64(13), object(2)
memory usage: 1.1+ MB


None

<b>infer_dtype():</b>

Date               string
Time               string
CO(GT)           floating
PT08.S1(CO)      floating
NMHC(GT)         floating
C6H6(GT)         floating
PT08.S2(NMHC)    floating
NOx(GT)          floating
PT08.S3(NOx)     floating
NO2(GT)          floating
PT08.S4(NO2)     floating
PT08.S5(O3)      floating
T                floating
RH               floating
AH               floating
dtype: object

## Multivariate Time Series (MTS)

A <b>multivariate time series</b> has more than one time-dependent variable just like our dataset. Each variable depends not only on its past values but also has some dependency on other variables. This dependency is used for forecasting future values.

### to_datetime()

In [10]:
data['Time'] = pd.to_datetime(data.Time, format='%H.%M.%S').dt.time
data['Time'].head()

0    18:00:00
1    19:00:00
2    20:00:00
3    21:00:00
4    22:00:00
Name: Time, dtype: object

In [11]:
data['Date_Time'] = pd.to_datetime(data['Date'].apply(str) + ' ' + data['Time'].apply(str))
data['Date_Time'].head()

0   2004-10-03 18:00:00
1   2004-10-03 19:00:00
2   2004-10-03 20:00:00
3   2004-10-03 21:00:00
4   2004-10-03 22:00:00
Name: Date_Time, dtype: datetime64[ns]

In [12]:
data = data.set_index('Date_Time')
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date_Time,,,,,,,,,,,,,,,
2004-10-03 18:00:00,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-10-03 19:00:00,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-10-03 20:00:00,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
2004-10-03 21:00:00,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
2004-10-03 22:00:00,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [13]:
data = data.drop(['Date', 'Time'], axis=1)
data.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date_Time,,,,,,,,,,,,,
2004-10-03 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-10-03 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-10-03 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
2004-10-03 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
2004-10-03 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


## Imputation

Since the missing values in the data are replaced with a value -200, we will have to impute the missing value with a better number. Consider this - if the present dew point value is missing, we can safely assume that it will be close to the value of the previous hour. Makes sense, right? Here, I will impute -200 with the previous value.

In [14]:
cols = data.columns

# iterate through all columns
for j in cols:
    # iterate through all rows
    for i in range(0, len(data)):
        if data[j][i] == -200:
            data[j][i] = data[j][i-1]

## Stationarity

A stationary time series will more often than not give us a better set of predictions. Similar to the <b>Augmented Dickey-Fuller test</b> for univariate series, we have <b>Johansen’s test</b> for checking the stationarity of any multivariate time series data. Since the test works for only 12 variables, I have randomly dropped in the next iteration, I would drop another and check the eigenvalues.

### coint_johansen()

`statsmodels.tsa.vector_ar.vecm.coint_johansen(endog, det_order, k_ar_diff)` - perform the Johansen cointegration test for determining the cointegration rank of a VECM. 
* <b>endog</b> - the data with presample.
* <b>det_order: int</b> -
  * <b>-1</b> - no deterministic terms.
  * <b>0</b> - constant term.
  * <b>1</b> - linear trend.
* <b>k_ar_diff: int, nonnegative</b> - number of lagged differences in the model.

In [24]:
#from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [25]:
#johan_test_temp = data.drop(['CO(GT)'], axis=1)
#coint_johansen(johan_test_temp, -1, 1).eig

## Train-Validation Split

Creating a validation set for time series problems is tricky because we have to take into account the time component. One cannot directly use the `train_test_split` or k-fold validation since this will disrupt the pattern in the series. The validation set should be created considering the date and time values.

In [18]:
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

## Vector Autoregression (VAR)

In a <b>VAR</b> model, each variable is a linear function of the past values of itself and the past values of all the other variables. Unlike AR, VAR is able to understand and use the relationship between several variables. This is useful for describing the dynamic behavior of the data and also provides better forecasting results. Additionally, implementing VAR is as simple as using any other univariate technique.

In [19]:
from statsmodels.tsa.vector_ar.var_model import VAR

/Users/mosthusnejahan/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### VAR()

`statsmodels.tsa.vector_ar.var_model.VAR(endog, exog=None, dates=None, freq=None, missing='none')` - fit VAR(p) process and do lag order selection.
\begin{equation*} 
y_t = A_1y_{t-1} + ... + A_py_{t-p} + u_t
\end{equation*}
* <b>endog</b> - 2-d endogenous response variable. The independent variable.
* <b>exog</b> - 2-d exogenous variable.
* <b>dates</b> - must match number of rows of endog

In [20]:
# Fit the model
model = VAR(endog=train)
model_fit = model.fit()

# Make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/Users/mosthusnejahan/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/vector_ar/var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]


In [21]:
# Convert array to DataFrame
cols = data.columns
pred = pd.DataFrame(index=range(0, len(prediction)), columns=[cols])

for j in range(0, len(cols)):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

In [22]:
# RMSE
for i in cols:
    print('RMSE value for', i, 'is:', sqrt(mean_squared_error(pred[i], valid[i])))

RMSE value for CO(GT) is: 1.408688883687256
RMSE value for PT08.S1(CO) is: 205.89558284021305
RMSE value for NMHC(GT) is: 6.67354871134285
RMSE value for C6H6(GT) is: 7.130087248705714
RMSE value for PT08.S2(NMHC) is: 277.84844376800464
RMSE value for NOx(GT) is: 214.7832234091224
RMSE value for PT08.S3(NOx) is: 244.9576966193527
RMSE value for NO2(GT) is: 66.69695211710588
RMSE value for PT08.S4(NO2) is: 490.08388934110286
RMSE value for PT08.S5(O3) is: 446.51541648881795
RMSE value for T is: 10.72132579560568
RMSE value for RH is: 17.111676248172948
RMSE value for AH is: 0.5216247245187544


In [23]:
# Make final predictions
model = VAR(endog=data)
model_fit = model.fit()

yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[2.34596328e+00 1.08633212e+03 2.80762173e+02 1.24130779e+01
  1.05535947e+03 2.80882233e+02 6.59534851e+02 1.68444418e+02
  1.15918056e+03 8.50845529e+02 2.73639014e+01 1.55311062e+01
  5.15317053e-01]]


/Users/mosthusnejahan/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/vector_ar/var_model.py:461: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(z, y_sample)[0]
